In [15]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from scipy import sparse
import scipy
import sys
sys.path.append('..')
from importlib import reload
from utils import optm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from importlib import reload
reload(optm)

<module 'utils.optm' from '/Users/keisukemurota/Documents/todo/worms/python/nsp/utils/optm.py'>

In [16]:
tmp = torch.ones(4)
x = torch.nn.Parameter(tmp)
y = loss(x)
optimizer = torch.optim.Adam([x], lr=0.2)
y.backward()
x.grad[0] = 0
optimizer.step()

### test new algorithm

In [17]:
def get_mat_status(X):
    X = np.array(X)
    L = X.shape[0]
    lps = int(np.sqrt(L))
    if lps**2 != L:
        print("! ---- This matrix might not be the bond operator ---- !")
    if not np.all(np.diag(X)>=0):
        print("! ---- diagonal elements of the local hamiltonian should be non negative ---- !")
        
    E, V = np.linalg.eigh(X)
    return L, lps, E

def optim_matrix_symm(X, N_iter, lr, optm_method = torch.optim.Adam, loss_func = optm.loss_eig):
    L, lps, E = get_mat_status(X)
    model = optm.unitary_solver([lps,lps],syms=True)
    optimizer = optm_method(model.parameters(), lr=lr)
    
    print("target loss : {:.2f}\n".format(E[-1]))
    
    print("-"*10, "iteration start", "-"*10)
    
    for t in range(N_iter):
        y = model(X)
        loss = loss_func(y)
        if t % 1000 == 0:
            print("iteration : {:4d}   loss : {:.3f}".format(t,loss.item()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

In [20]:
L = 5
X = torch.rand((L**2,L**2),dtype=torch.float64)*2-1
X = X + X.T + torch.eye(L**2)*2

In [27]:
reload(optm)
model = optm.optim_matrix_symm(X, 5000, 0.001)

target loss : 8.22

---------- iteration start ----------
iteration :    0   loss : 16.294
iteration : 1000   loss : 14.992
iteration : 2000   loss : 14.992
iteration : 3000   loss : 14.992
iteration : 4000   loss : 14.992


 -------------- results --------------
target loss      : 8.22
loss before optm : 16.76
loss after optm  : 14.99
